<a href="https://colab.research.google.com/github/mosomo82/COMP_SCI5501/blob/main/Assignment/Assignment2_Searching_Algorithm/src/Assignment2_5501_Searching_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import time   # libary for calculating the execution time
import random # libary for generating the key randomly
import pandas as pd
import plotly.express as px
import requests # <-- New library for fetching data from URL

# ==============================================================================
# 1. SEARCH ALGORITHMS (Functions are unchanged)
# ==============================================================================

def linear_search(data, key):
    """Performs a standard linear search and count the number of comparisons"""
    comparisons = 0
    for i in range(len(data)):
        comparisons += 1
        if data[i] == key:
            return i, comparisons
    return -1, comparisons

def sentinel_search(data, key):
    """Performs a sentinel search and count number of comparisons"""
    n = len(data)
    last_element = data[n - 1]
    data[n - 1] = key
    comparisons = 0
    i = 0
    while True:
        comparisons += 1
        if data[i] == key:
            break
        i += 1

    data[n - 1] = last_element

    if i < n - 1 or data[n - 1] == key:
        return i, comparisons
    else:
        return -1, comparisons

def binary_search(data, key):
    """Performs an iterative binary search and count number of comparisons."""
    low = 0                            # key in order list
    high = len(data) - 1               # Look between low and high
    comparisons = 0                    # Initialize comparisons
    while low <= high:
        mid = (low + high) // 2        # Select the mid point
        comparisons += 1               # Count the comparisons
        if data[mid] == key:           # Did we find key at midpoint?
            return mid, comparisons    # Return the location of key and the total of comparisons
        elif data[mid] < key:          # Is key in upper half?
            low = mid + 1              # Yes, raise the low boundary
        else:
            high = mid - 1             # No, but could be in lower half
    return -1, comparisons             # Key not found and return the total of comparisons

def ternary_search(data, key):
    """Performs an iterative ternary search and count number of comparisons."""
    low = 0
    high = len(data) - 1
    comparisons = 0
    while low <= high:
        # Divide the range into 3 parts
        mid1 = low + (high - low) // 3
        mid2 = high - (high - low) // 3

        # Check if the key is at mid1 or mid2
        comparisons += 1
        if data[mid1] == key:
            return mid1, comparisons

        comparisons += 1
        if data[mid2] == key:
            return mid2, comparisons

        # Narrow down the search space
        comparisons += 1
        if key < data[mid1]:
            high = mid1 - 1
        elif key > data[mid2]:
            low = mid2 + 1
        else:
            low = mid1 + 1
            high = mid2 - 1

    return -1, comparisons                  # the key is not found, return comparisons

# ==============================================================================
# 2. UPDATED DATA READING & EXPERIMENT SETUP
# ==============================================================================

# Task 1: Read the given text file and store the string data in Python collection as
# linear data container
def read_data_from_url(url):
   response = requests.get(url)
   data = response.text.splitlines()
   return data

# return the result of experiments in a data frame which has type of searching
# algorithms, number of treatment , execution time and comparisons
def run_experiment(data, num_treatments=20):
    """Runs the full experiment for all algorithms."""
    results = []
    print(f"Generating {num_treatments} random search keys from the data...")

    # Task 2: Generate the key randomly
    keys_to_search = [random.choice(data) for _ in range(num_treatments)]

    # show the random keys selected
    print("Random keys selected:")
    print(keys_to_search)


    print("Running treatments...")
    for i in range(num_treatments):
        key = keys_to_search[i]
        treatment_num = i + 1

        # Task 6: Calculate the execution time as physical time using RTC and convert
        # to miliseconds by mulipplied by 1e6

        # --- Linear Search ---
        start_time = time.time()
        _, comps_linear = linear_search(data, key)
        end_time = time.time()
        execution_time_linear = (end_time - start_time) * 1e6
        results.append(('Linear', treatment_num, execution_time_linear, comps_linear))

        # --- Sentinel Search ---
        start_time = time.time()
        _, comps_sentinel = sentinel_search(data.copy(), key)
        end_time = time.time()
        execution_time_sentinel = (end_time - start_time) * 1e6
        results.append(('Sentinel', treatment_num, execution_time_linear, comps_sentinel))

        # --- Binary Search ----
        start_time_sort = time.time()
        # Sort the data and the cost is inclusive
        binary_sorted = sorted(data)
        _, comps_binary = binary_search(binary_sorted, key)
        end_time_search = time.time()
        execution_time_binary = (end_time_search - start_time_sort) * 1e6
        results.append(('Binary', treatment_num, execution_time_binary, comps_binary))

        # --- Ternary Search ---
        start_time_sort = time.time()
        # Sort the data and the cost is inclusive
        ternary_sorted = sorted(data)
        _, comps_ternary = ternary_search(ternary_sorted, key)
        end_time_search = time.time()
        excecution_time_ternary = (end_time_search - start_time_sort) * 1e6
        results.append(('Ternary', treatment_num, excecution_time_ternary, comps_ternary))

    print("Experiment complete.")
    return pd.DataFrame(results, columns=['Algorithm', 'Treatment', 'Execution Time (µs)', 'Comparisons'])

# ==============================================================================
# 3. MAIN EXECUTION AND ANALYSIS
# ==============================================================================

if __name__ == "__main__":
    # The URL for the raw text file
    url = 'https://raw.githubusercontent.com/mosomo82/COMP_SCI5501/refs/heads/main/Assignment/Assignment2_Searching_Algorithm/raw_data/words_alpha.txt'

    try:
        print(f"Fetching data from URL: {url}...")
        main_data = read_data_from_url(url)
        print(f"Successfully loaded {len(main_data)} words.")

    except requests.exceptions.RequestException as e: # catch if the link is invalid
        print(f"Error: Could not fetch data from the URL. Please check the link and your connection.")
        print(f"Details: {e}")
        exit()


 # for each algorithm execute this process twenty times

    df_results = run_experiment(main_data, num_treatments=20)


    # Task 7: Show the comparative analysis for each treatment
    # i. In form of tabulated data
    print("\n--- Comparative Analysis for Each Treatment ---")
    df_pivot_time = df_results.pivot(index='Treatment', columns='Algorithm', values='Execution Time (µs)')
    print("\nExecution Time (µs) per Treatment:")
    print(df_pivot_time.round(2))

    df_pivot_comps = df_results.pivot(index='Treatment', columns='Algorithm', values='Comparisons')
    print("\nNumber of Comparisons per Treatment:")
    print(df_pivot_comps)

    # ii. Interactive Line Charts
    print("\nGenerating interactive line charts...")

    fig_line_time = px.line(df_results, x='Treatment', y='Execution Time (µs)', color='Algorithm',
                            title='Interactive: Execution Time per Treatment', markers=True)
    fig_line_time.show()

    fig_line_comps = px.line(df_results, x='Treatment', y='Comparisons', color='Algorithm',
                             title='Interactive: Number of Comparisons per Treatment', markers=True)
    fig_line_comps.show()

    # Task 8: show  comparative analysis as average time for each algorithm
    # i. In form of tabluted data
    summary_table = df_results.groupby('Algorithm').mean(numeric_only=True).drop(columns='Treatment')
    print("\n--- Average Performance Analysis ---")
    print(summary_table.round(2))

    # ii: Interactive Bar Charts
    print("\nGenerating interactive bar charts...")
    summary_table = summary_table.reset_index()

    fig_bar_time = px.bar(summary_table, x='Algorithm', y='Execution Time (µs)', color='Algorithm',
                          title='Interactive: Average Execution Time')
    fig_bar_time.show()

    fig_bar_comps = px.bar(summary_table, x='Algorithm', y='Comparisons', color='Algorithm',
                           title='Interactive: Average Number of Comparisons')
    fig_bar_comps.show()




Fetching data from URL: https://raw.githubusercontent.com/mosomo82/COMP_SCI5501/refs/heads/main/Assignment/Assignment2_Searching_Algorithm/raw_data/words_alpha.txt...
Successfully loaded 370104 words.
Generating 20 random search keys from the data...
Random keys selected:
['outcursing', 'homousian', 'dod', 'lopes', 'hypergeneticalness', 'taeniae', 'unlives', 'carbolfuchsin', 'exaltations', 'crones', 'antiarcha', 'coelenteric', 'scores', 'pedometers', 'fussiest', 'ethenyl', 'sitz', 'nicenian', 'hypertropical', 'thiosulphonic']
Running treatments...
Experiment complete.

--- Comparative Analysis for Each Treatment ---

Execution Time (µs) per Treatment:
Algorithm    Binary    Linear  Sentinel   Ternary
Treatment                                        
1          39435.39  29176.47  29176.47  36010.50
2          44135.81  22031.55  22031.55  44972.18
3          41384.94  11262.18  11262.18  41149.38
4          40789.84  20027.64  20027.64  43843.75
5          42094.71  18064.98  18064.98 


--- Average Performance Analysis ---
           Execution Time (µs)  Comparisons
Algorithm                                  
Binary                28874.34        17.75
Linear                13581.62    171076.20
Sentinel              13581.62    171076.20
Ternary               28039.91        31.45

Generating interactive bar charts...


 iii. Based on the observation:

*   **Execution time:** Linear search and sentinel serach have way lower average execution times compared to binary search and ternary search. The reason is the cost of sorting data are included for binary and ternary search. By its mean, the cost of sorting step is expensive and it dominate the search time for this experiment setup.
*   **Comparisons:** Binary and ternary serach have very lower average number of comparisons when comparing to linear and sentinel search. The reason is the algorithm of binary and ternary search. They divide the search space in the large **sorted** dataset.

In summary, binary and ternary serach are much more efficient in terms of the number of comparisons for sorted data.


In [5]:
import requests
import random
import pandas as pd

# 1. Read the given text file and store the string data
def get_and_structure_data(url):
    """Fetches data from a URL and structures it into a 1D list and a 2D list."""
    print(f"Fetching data from {url}...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        all_words = response.text.splitlines()
        print(f"Successfully loaded {len(all_words)} words.")
    except requests.exceptions.RequestException as e: # catch if the link is invalid
        print(f"Error: Could not fetch data from the URL. Please check the link and your connection.")
        print(f"Details: {e}")
        exit()

    # 1. Create the 1D List
    linear_list = all_words

    # 2. Create the 2D List
    # Initialize 26 empty sub-lists, one for each letter of the alphabet
    list_2d = [[] for _ in range(26)]

    for word in all_words:
        if word:                                      # Ensure the word is not an empty string
            first_letter = word[0].lower()            # get the character of the word and convert to lowercase
            if 'a' <= first_letter <= 'z':            # Check if the character is a standard letter
                index = ord(first_letter) - ord('a')  # Caclulate the index for 2D list
                list_2d[index].append(word)

    return linear_list, list_2d

def linear_search_1d(data, key):
    """Performs a standard linear search on a 1D list and count the comparisons."""
    comparisons = 0
    for item in data:
        comparisons += 1
        if item == key:
            return comparisons
    return comparisons

def linear_search_2d(data_2d, key):
    """Performs a linear search on the appropriate sub-list of a 2D list and count the comparisons."""
    comparisons = 0
    if not key:
        return 0

    # Step 1: Find the correct sub-list (bucket)
    first_letter = key[0].lower()             # get the first character of the key and convert to lowercase
    if 'a' <= first_letter <= 'z':
        index = ord(first_letter) - ord('a')
        sub_list = data_2d[index]            # Create a approciate sub-list based on the first character of the key

        # Step 2: Perform linear search only on the smaller sub-list. One of 26 lists within
        # data_2d list.
        for item in sub_list:               # iterate through each item in correct sub_list
            comparisons += 1
            if item == key:
                return comparisons
        return comparisons
    else:
        # Key does not start with a standard letter
        return 0

# --- Main Execution ---
if __name__ == "__main__":
    DATA_URL = 'https://raw.githubusercontent.com/mosomo82/COMP_SCI5501/refs/heads/main/Assignment/Assignment2_Searching_Algorithm/raw_data/words_alpha.txt'
    NUM_TREATMENTS = 10

    # Task 1: Read text data file from URL  and structure the data into 1D list and 2D list
    linear_data, data_2d = get_and_structure_data(DATA_URL)

    if linear_data:
        # Task 2: Generate 10 keys and run the experiment
        results = []  # create a results list
        # store 10 random keys into a list generated from 1D list
        search_keys = [random.choice(linear_data) for _ in range(NUM_TREATMENTS)]

        print(f"\nRunning {NUM_TREATMENTS} search treatments...")
        for i in range(NUM_TREATMENTS):
            key = search_keys[i]                    # Get the i-th search key from the search_keys list

            # Perform  a linear search on 1D & 2D list to find the key and return the number of comparisons
            # These numbers are stored in the approriate 1D & 2D variables.
            comps_1d = linear_search_1d(linear_data, key)
            comps_2d = linear_search_2d(data_2d, key)

            # Use a dictionary to store the values into the results list
            results.append({
                'Treatment': i + 1,
                'Search Key': key,
                'Comparisons (1D List)': comps_1d,
                'Comparisons (2D List)': comps_2d
            })

        # Task 3: Tabulate results and calculate the average
        df = pd.DataFrame(results)

        print("\n--- Comparison Results per Treatment ---")
        print(df)

        print("\n--- Average Number of Comparisons ---")
        print(df[['Comparisons (1D List)', 'Comparisons (2D List)']].mean())

Fetching data from https://raw.githubusercontent.com/mosomo82/COMP_SCI5501/refs/heads/main/Assignment/Assignment2_Searching_Algorithm/raw_data/words_alpha.txt...
Successfully loaded 370104 words.

Running 10 search treatments...

--- Comparison Results per Treatment ---
   Treatment          Search Key  Comparisons (1D List)  Comparisons (2D List)
0          1         parasitidae                 226101                   3564
1          2          polydental                 241819                  19282
2          3  nonsophisticalness                 207405                  11008
3          4           poroscopy                 243499                  20962
4          5               smerk                 294839                  18866
5          6           daggering                  76201                    264
6          7     nomographically                 200929                   4532
7          8         cartoonists                  48667                   4837
8          9      

4.  Based on the comparison results for linear search on the 1D list versus the 2D list. This shows that choosing the right data structure choice can greatly impact the algorithm performance. Structuring the data into a 2D list based on the first letter eliminates unnecessary search space and focuses on searching on the smaller sub-list. As a result of the above, a 2D list only requires an average of 13,000 comparisons, which is way lower than that of a 1D list having an average of 190,000 comparisons.  





 structuring the data into a 2D list based on the first letter significantly improves the efficiency of linear search by reducing the search space. Instead of having to potentially scan the entire list, the search is narrowed down to a much smaller sub-list. This is a practical example of how data structure choice can impact algorithm performance.